# Transformer 优化实现

在本教程的初始实现中，我们构建了一个基础版Transformer模型（见第1部分），然而该基线模型在工业级应用中面临显著挑战：其训练阶段和推理阶段存在计算密集型操作，推理时遭遇内存瓶颈，且存在数值溢出的风险。为此，我们将采用一部分学术界及开源社区中常见的策略，对模型架构和计算范式进行系统性优化。

本文的核心讨论聚焦于算法代码实现层面，针对优化机理将不展开系统性论述，但会在相应章节为深度技术解析提供参考资料。

---

## 准备工作

---

引入必要的库和常量

In [ ]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.autograd import Variable
import math

## Attention 优化

---

正如经典论文[《Attention is all you need》](https://arxiv.org/abs/1706.03762)所阐述的，Attention模块是Transformer架构的核心。优化该模块可显著提升整个Transformer神经网络的性能。以下为常见的优化策略：

- 实现 Multi-Head Attention 的并行化运算；
- 引入 KV-Cache 机制。

自 2017 年 Transformer 发表以来，学术领域及开源社区出现了多种优化方案。例如，2022年发表的论文[《FlashAttention: Fast and Memory-Efficient Exact Attention with IO-Awareness》](https://arxiv.org/abs/2205.14135)介绍了一种名为 “FlashAttention” 的结构，可显著的加速注意力计算并减少内存占用。然而， FlashAttention 的实现更接近于对矩阵运算底层的优化（如硬件IO分配和分块计算策略），而非在注意力机制整体结构上进行的优化，我们对于此类的优化暂不讨论。

事实上尽管开源社区有着很多更好的优化算法，但我们仍需关注上述的两种优化策略，是因为在 LLM 开源社区（如 llama、deepseek 等）中，它们及它们的变体实现被广泛采用。为了更好地分析开源 LLM 的实现思路，我们需要复现这两种优化策略。

### 1. Multi-Head Attention 并行化运算

在基础版本的 Transformer 实现中，我们先构建了 Scaled Dot-product Attention 模块（以下简称 “Attention 模块”），再将其整合到 Multi-Head Attention 模块中。在运算过程中，通常以串行方式逐一获取结果张量，最后将这些张量拼接成一个大的张量。这种实现方式虽然逻辑清晰，但在实际运行中却存在明显的低效问题，主要体现在以下两个方面：

- 运行时需要将单个 Attention 模块中的数据逐一加载入内存，由于其内存不连续就会导致通讯成本高、内存占用大。
- 运算采用串行方式，无法进行并行运算，导致运算速度较慢。

首先，我们将原来单个 Attention 模块的 $W_{q}$ 、$W_{k}$ 、$W_{v}$ 根据所需的头数直接合并为一个大的张量，如下图所示（仅以 $W_{q}$ 为例，$W_{k}$ ，$W_{v}$ 类同）：

![Multi-Head Attention Weight](img/Multi-Head-Attention-Weight-Q.png)

这样我们就可以一次性的将 Multi-Head Attention 中所有的权重一次性的加载到了内存中，省去了通讯成本，同时由于不用再保存单个 Attention 模块的信息还节约了内存占用。

我们将多个 Attention 的权重合并为一个大的权重后，其进行矩阵乘法运算时如下图所示：

![Multi-Head Attention Weight Matmul](img/Multi-Head-Attention-Weight-Matmul-Q.png)

可见矩阵乘法并不会改变其原始的 Attention 权重逐个相乘最后再合并后的结果，并且我们通过上述方法也达成了并行计算的目标。

### 2. KV Cache

KV Cache是一种缓存机制，通过存储 Transformer 模型自回归生成过程中已计算的 Key 和 Value 矩阵，避免在后续生成新的 token 时重复计算这些值，从而提高推理效率。其中 KV Cache 的要求如下：

- KV Cache 只在多个 token 生成步骤中发生，并且**仅在decoder进行**。
- KV Cache 需要保持 $W_{k}$ 和 $W_{v}$ 不变，即其**仅在推理时进行**。

其原理如下图所示：

![KV Cache](img/KV-Cache.png)

对于传入序列 $X$ ，根据矩阵乘法，对于第 $i$ 行的词嵌入向量 $x_{i}$ 都与 $W_{q}/W_{k}/W_{v}$ 权重矩阵相乘，其结果就是 $Q/K/V$ 对应的第 $i$ 行的向量。如果在 $X$ 序列尾部再增加一行 $x_{last}$ ，那么只需在最终的 $Q/K/V$ 结果中再加一行 $x_{last}$ 与 $W_{q}/W_{k}/W_{v}$ 权重相乘结果的向量即可。所以这里 KV Cache 就是为了缓存之前的计算好的 $K$ 和 $V$ 的。

为什么仅缓存 $K$ 和 $V$ 而不缓存 $Q$ ，因为我们只关注最后一个预测的 token。

推荐观看：[知乎-看图学KV Cache](https://zhuanlan.zhihu.com/p/662498827)

### 3. 优化的多头注意力模块代码实现

注意：使用优化后的多头注意力需要同时对对应的 Encoder 和 Decoder 进行修改，请参考 transformer.py 中的代码

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, dim_emb:int, dim_head:int, dim_out:int, head_num:int):
        """
        Multi-Head Attention

        :param dim_emb: 数据的嵌入维度
        :param dim_head: 每个 Attention 块对应的头维度
        :param dim_out: 数据的输出维度
        :param head_num: 注意力头数
        """
        super().__init__()

        self.dim_emb = dim_emb
        self.dim_head = dim_head
        self.head_num = head_num

        # 计算多个 Attention 头维度合并后的维度
        dim_mid = dim_head * head_num

        # 创建 Attention 权重
        self.Q_net = nn.Linear(dim_emb, dim_mid)
        self.K_net = nn.Linear(dim_emb, dim_mid)
        self.V_net = nn.Linear(dim_emb, dim_mid)

        # 创建权重
        self.W_net = nn.Linear(dim_mid, dim_out)

    def forward(self, x_q, x_k, x_v, mask_mat = None, use_cache = False, kv_cache = None):

        # 获取各个维度的值
        batch, q_seq_len, _ = x_q.size()
        _, kv_seq_len, _ = x_k.size()

        # 求 q、k、v 的值，并将其形状变为 (batch, head_num, seq_len, dim_head)
        # 注意这里不能直接进行 view 操作，会导致张量中的元素分配错误，具体详情请参考 pytorch 官方文档
        q = self.Q_net(x_q).view(batch, q_seq_len, self.head_num, self.dim_head).transpose(1, 2)
        k = self.K_net(x_k).view(batch, kv_seq_len, self.head_num, self.dim_head).transpose(1, 2)
        v = self.V_net(x_v).view(batch, kv_seq_len, self.head_num, self.dim_head).transpose(1, 2)

        # 如果使用 KV Cache，拼接历史 K 和 V
        if use_cache and kv_cache is not None:
            k_prev, v_prev = kv_cache
            # 在序列长度维度拼接
            k = torch.cat([k_prev, k], dim=2)
            v = torch.cat([v_prev, v], dim=2)

        # 记录新的 KV Cache（供下一步使用）
        new_kv_cache = (k, v) if use_cache else None

        # 将 K 的最后两个维度进行转置，转置后的维度为 (batch, head_num, dim_head, kv_seq_len)
        k_t = k.transpose(-1, -2)

        # 计算 qk^T / sqrt(d_k)，此时 s0 的维度为 (batch, head_num, q_seq_len, kv_seq_len)
        s0 = torch.matmul(q, k_t) / math.sqrt(self.dim_head)

        # 进行掩码遮掩操作
        if mask_mat is not None:
            # 进行遮掩
            s0 = torch.masked_fill(s0, mask_mat, float('-inf'))

        # 计算 softmax(s)*v ，此时 s1 的维度为 (batch, head_num, q_seq_len, dim_head)
        s1 = torch.matmul(F.softmax(s0, dim=-1), v)

        # 我们需要使用 s1*W ，这里就要将矩阵变换为可以跟 W 矩阵进行矩阵乘法的维度，即：
        # s1 变换维度为：(batch, q_seq_len, dim_head * head_num)
        s1 = s1.transpose(1, 2).contiguous() # 这里需要让内存连续（ 使用 reshape 则不用）
        s1 = s1.view(batch, q_seq_len, self.head_num * self.dim_head)

        # 输出的最终维度为：(batch, q_seq_len, dim_out)
        output = self.W_net(s1)

        return output, new_kv_cache

**测试 MultiHeadAttention**

In [ ]:
# 配置
# =====================================
batch = 3
seq_len = 8
dim_emb = 6
head_num = 2
dim_head = dim_emb // head_num
dim_out = dim_emb
# =====================================

attention = MultiHeadAttention(dim_emb, dim_head, dim_emb, head_num)

# ===============
#  模拟第一次调用
# ===============

# 模拟第一次传入的 X，这个 seq_len 可以不用限制
X = torch.randn((batch, seq_len, dim_emb))

# 生成不包含对角线的上三角矩阵（设置 diagonal=1）
mask_mat = torch.triu(torch.ones(seq_len, seq_len, dtype=torch.bool), diagonal=1)

# 第一次输出
output, kv_cache = attention(X, X, X, mask_mat, use_cache = True)
k_cache, v_cache = kv_cache

# 打印输出及 KV Cache 维度信息
print(f'output size : {output.size()}')
print(f'k cache size : {k_cache.size()}')
print(f'v cache size : {v_cache.size()}')

# ===============
#  模拟第二次调用
# ===============

# 模拟生成下一个 token
# 注意此时需要 seq = 1，因为使用 KV Cache 只需传入下一个生成的 token 即可
X_next = torch.randn(batch, 1, dim_emb)

# 第二次调用（携带缓存），此时不用传入掩码矩阵
output_next, kv_cache = attention(X_next, X_next, X_next, use_cache=True, kv_cache=kv_cache)
k_cache_next, v_cache_next = kv_cache

# 打印第二次调用的输出及 KV Cache 维度信息
print(f'output size : {output_next.size()}')
print(f'next k cache size : {k_cache_next.size()}')
print(f'next v cache size : {v_cache_next.size()}')

### 4. 其他 Attention 优化变种

#### MQA（Multi-Query Attention，多查询注意力）

- **特征：** MQA 是 MHA （Multi-Head Attention 缩写） 的一种优化，其核心思想是**让所有的 Query 头共享同一组 Key 和 Value**，而每个 Query 头保留自己独特的 Query 参数。这样，所有 Query 头都基于相同的 Key 和 Value 进行学习，减少了冗余计算和参数量。
- **优点：**
    - 极大减少计算成本和内存占用，适用于推理阶段，特别是处理超长文本或大规模推理任务时，能显著降低内存需求。
    - 解码速度显著提升，让模型响应更快。
- **缺点：**
可能损失部分表达能力，仅有一个 Key-Value 可能影响模型对不同 Query 头语义信息的区分度，从而在一定程度上降低模型性能。

#### GQA（Grouped-Query Attention，分组查询注意力）

- **特征：** GQA 是 MQA 的进一步扩展，**它将 Query 头分成若干组，每组内的 Query 头共享同一组 Key 和 Value**，而不同组之间则拥有独立的 Key 和 Value。这样在计算效率和表达能力之间找到了更好的平衡点。
- **优点：**
    - 计算量和内存占用较 MHA 有显著降低，同时比 MQA 具有更好的表达能力，性能更接近 MHA，通常能达到 MHA 性能的 98%-99%。
    - 在长文本处理和大规模数据任务中表现出色，效率高且能保持较好的模型效果。
- **缺点：**
实现上略微复杂，需要对 Query 头进行分组处理。
相比 MHA，在某些任务上可能会有轻微的性能下降，但通常在可接受范围内。

目前 **GQA 应用较为广泛** 如 llama3 、Qwen-1.5-32B 等，推荐观看：[MHA、MQA、GQA各种注意力变种机制讲解](https://www.bilibili.com/video/BV17CPkeEE5d)

## Dynamic Tanh

---

何恺明、Yann LeCun及刘壮团队近期提出的 **Dynamic Tanh（DyT）** 算法，通过简单的 `tanh` 函数实现了对Transformer架构中归一化层的替代，由于减少了大量的指数和求和运算，使其在性能、效率和成本上展现了显著优势。这一研究挑战了深度学习领域长期以来的固有观念，即归一化层（如LayerNorm、RMSNorm）是神经网络训练不可或缺的组件。

论文地址：[Transformers without Normalization](https://arxiv.org/abs/2503.10622)

### 1. 技术介绍

研究团队通过分析 Vision Transformer（ViT）、语音模型wav2vec 2.0和扩散模型DiT等架构中的归一化层发现，**层归一化（LayerNorm）的输出与输入之间呈现类似`tanh`函数的 S 形曲线** 。尽管 LayerNorm 本质上是线性操作（减去均值、除以标准差），但不同 token 或通道的统计量差异导致整体映射呈现非线性特征。这种非线性主要体现在对极端值的压缩上，例如将输入值大于 50 或小于 -50 的极端值映射到更温和的范围内。

基于上述观察，团队提出直接使用 **动态调整的tanh函数（DyT）** 替代归一化层，其数学表达式为：

$$
DyT(x) = tanh(\alpha x) \cdot \gamma + \beta
$$

其中：

- $\alpha$：可学习的标量参数，控制输入缩放；
- $\gamma$ 和 $\beta$ ：可学习的通道级仿射参数（与归一化层中的参数类似），用于调整输出的尺度和偏移。

### 2. 代码实现

使用方法就是只接用下面的代码替换 transformer 中的 RMSNorm 即可。

In [ ]:
class DyT(nn.Module):
    def __init__(self, dim):
        """
        Dynamic Tanh

        :param dim: 维度
        """
        super().__init__()

        self.alpha = nn.Parameter(torch.ones(1))
        self.weight = nn.Parameter(torch.ones(dim))
        self.bias = nn.Parameter(torch.zeros(dim))
    def forward(self, x):
        x = torch.tanh(self.alpha * x)
        return x * self.weight + self.bias

## 前馈神经网络

---

### SwiGLU

前馈神经网络一般采用 SwiGLU（Swish-Gated Linear Unit）来实现，SwiGLU是一种结合了Swish和GLU（门控线性单元）两种激活函数优点的新型激活函数，于2020年由谷歌提出。它在大型语言模型（如LLaMA、OLMO和PALM）中被广泛采用，表现出色。其公式如下：

$$
SwiGLU(x,W,V) = Swish_{\beta}(xW)\otimes(xV)
$$

其中 $x$ 是输入，$W$ 和 $V$ 是权重矩阵，$\beta$ 是一个可学习的参数，$\otimes$ 表示逐个元素相乘。Swish函数的公式为：

$$
Swish_{\beta} = x \cdot sigmoid (\beta x)
$$

其中，当 $\beta=1$ 时， Swish 函数等价于 SiLU（Sigmoid Linear Unit）函数。（在前馈神经网络实际实现时，我们会令 $\beta = 1$）

注意， SwiGLU 只是激活函数，我们实际实现的是一个两层的神经网络，即还需要乘上第二层神经网络连接的权重 $W_{2}$ 才是最终返回结果  ，即：

$$
FNN_{SwiGLU}(x) = SwiGLU(x,W,V) W_{2}
$$

#### 关于 multiple_of 参数

在 LLaMA 模型实现的 SwiGLU 前馈神经网络中，有一个 `multiple_of` 参数，该参数的作用是确保隐藏层的大小是某个特定值的倍数。这个参数通常用于优化计算性能和内存使用:

- **确保隐藏层大小是特定值的倍数：** 在神经网络中，隐藏层的大小会影响计算效率和内存占用。通过设置 `multiple_of` 参数，可以确保隐藏层的大小是某个特定值的倍数，比如 `256` 。这样做的好处是，可以更好地利用硬件资源，比如在 GPU 上进行矩阵运算时，数据的维度要是 2 的幂次方会更高效。
- **优化计算性能和内存使用：** 通过确保隐藏层的大小是特定值的倍数，可以减少计算过程中的冗余操作，提高计算效率。同时，这也可能有助于减少内存碎片，提高内存的使用效率。

有关 `multiple_of` 参数的更多细节可以参考：[chaofa用代码打点酱油-swiglu-的表达形式](https://bruceyuan.com/llms-zero-to-hero/activate-function-from-relu-gelu-to-swishglu.html#_3-3-swiglu-%E7%9A%84%E8%A1%A8%E8%BE%BE%E5%BD%A2%E5%BC%8F)

推荐观看：[CSDN-SwiGLU 激活函数](https://blog.csdn.net/m0_53162279/article/details/142830585) \
相关论文：[GLU Variants Improve Transformer](https://arxiv.org/abs/2002.05202)

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, dim, dim_hidden):
        super().__init__()
        self.w1 = nn.Linear(dim, dim_hidden, bias=False)
        self.w2 = nn.Linear(dim_hidden, dim, bias=False)
        self.w3 = nn.Linear(dim, dim_hidden, bias=False)

    def forward(self, x):
        return self.w2(F.silu(self.w1(x)) * self.w3(x))

## Transformer 中常见模块的典型实现

---

在初版实现中，我们基于可读性优先原则，有意规避了复杂语法特性与高阶实现模式。这种设计选择虽能确保代码结构的直观性，但经架构评审发现存在两方面局限：1) 跨平台适配能力不足 2) 功能扩展点缺失。为提升工程化水平，本节将采用一些学术界及工程界常见写法来对之前的模块进行重构。

### 1. Mask Matrix Generate Function

我们观察掩码矩阵的形状，发现其是一个去掉主对角线的上三角矩阵，例如：

$$
M_{mask} = \begin{bmatrix}
0 & 1 & 1 \\
0 & 0 & 1 \\
0 & 0 & 0
\end{bmatrix}
$$

而 pytorch 提供了生成上三角矩阵的函数：`torch.triu(input, diagonal)`

- **input**：输入的原始矩阵
- **diagonal**：控制从哪条对角线开始提取上三角部分。

下面为测试代码：

In [ ]:
seq_len = 3

X = torch.randn((seq_len, seq_len))
print(f'input : \n {X}')

# 生成掩码矩阵
mask_mat = torch.triu(torch.ones(seq_len, seq_len, dtype=torch.bool), diagonal=1)
print(f'mask_mat : \n {mask_mat}')

masked_X = torch.masked_fill(X, mask_mat, float('-inf'))
print(f'masked_X : \n {masked_X}')

事实上我们在训练 Transformer 模型时，使用的数据序列一般都是长度不一的，比如有一个简单的中英翻译训练集：

|         English         | Chinese |
|:-----------------------:|:-------:|
|      How are you ?      |  你好吗？   |
| I am fine , Think you ! | 我很好，谢谢！ |
|      Good morning!      |  早上好!   |
|           ...           | ... |

很显然，无论是英语对应的句子长度还是中文对应的长度，二者都不是等长的，但是在训练时，我们是以“批”为训练单位的，每批内的序列长度都要保持一致，那么我们就需要想办法将每批内的数据进行填充，使其保持一致，如使用 `<pod>` 将`How are you ?` 填充为 `How are you ?<pod><pod><pod>`，使其长度与 `I am fine , Think you !` 保持一致。

但是我们希望神经网络的注意力集中在有意义的 `How are you ?` 上，而不是 `<pod>`，所以我们还需要对类似于 `<pod>` 的这些填充词进行遮掩，对于有效长度为 4，实际长度为 7 的注意力矩阵（1 代表有效信息）如下所示：

$$
M_{attention} = \begin{bmatrix}
1 & 1 & 1 & 1 & 0 & 0 & 0\\
\end{bmatrix}
$$

再配合上上三角遮掩矩阵，有：

$$
M_{mask} = \begin{bmatrix}
0 & 1 & 1 & 1 & 1 & 1 & 1\\
0 & 0 & 1 & 1 & 1 & 1 & 1\\
0 & 0 & 0 & 1 & 1 & 1 & 1\\
0 & 0 & 0 & 0 & 1 & 1 & 1\\
0 & 0 & 0 & 0 & 1 & 1 & 1\\
0 & 0 & 0 & 0 & 1 & 1 & 1\\
0 & 0 & 0 & 0 & 1 & 1 & 1\\
\end{bmatrix}
$$


In [ ]:
seq_len = 7

X = torch.randn((seq_len, seq_len))
print(f'input : \n {X}')

# 句子有效信息遮掩矩阵
mask_8_5 = torch.tensor([1, 1, 1, 1, 0, 0, 0], dtype=torch.bool)

# 生成掩码矩阵
mask_mat = torch.triu(torch.ones(seq_len, seq_len, dtype=torch.bool), diagonal=1)

# 使用广播机制进行广播
mask_mat = ~ mask_8_5.bool() | mask_mat

print(f'mask_mat : \n {mask_mat}')

masked_X = torch.masked_fill(X, mask_mat, float('-inf'))
print(f'masked_X : \n {masked_X}')

**多批次的掩码矩阵生成**

In [ ]:
batch = 3
seq_len = 4

X = torch.randn((batch , 2, seq_len, seq_len))
L_mask = torch.tensor([
    [1, 0, 0, 0],
    [1, 1, 1, 0],
    [1, 1, 0, 0]
])

t_mask_mat = torch.triu(torch.ones(seq_len, seq_len, dtype=torch.bool), diagonal=1)

mask_mat = torch.stack([t_mask_mat | (~ L_mask[i].bool()) for i in range (3)]).type(dtype=torch.bool)
print(f'mask_mat : \n {mask_mat}')
print(f'mask_mat size : \n {mask_mat.size()}')

masked_X = torch.masked_fill(X, mask_mat.view(batch, 1, seq_len, seq_len), float('-inf'))
print(f'masked_X : \n {masked_X}')

### 2. Positional Encoding

下面的代码使用了一些较为高级的语法，但本质与基础版本的实现是一致的。

In [ ]:
class PositionalEmbedding(nn.Module):
    def __init__(self, dim_emb: int, max_len: int = 5000):
        """
        正余弦位置编码模块

        :param dim_emb: 嵌入维度
        :param max_len: 预设最大序列长度 (default: 5000)
        """
        super().__init__()

        # 初始化位置编码矩阵 [max_len, dim_emb]
        pe = torch.zeros(max_len, dim_emb)

        # 生成位置序列 [max_len, 1]
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)

        # 计算频率调节因子 [dim_emb/2]
        div_term = torch.exp(
            torch.arange(0, dim_emb, 2).float() * (-math.log(10000.0) / dim_emb)
        )

        # 交替填充正弦余弦值
        pe[:, 0::2] = torch.sin(position * div_term)  # 偶数维度
        pe[:, 1::2] = torch.cos(position * div_term)  # 奇数维度

        # 注册为缓冲区 (不参与梯度计算)
        self.register_buffer('pe', pe.unsqueeze(0))  # [1, max_len, dim_emb]

    def forward(self, x, start, end) -> torch.Tensor:
        # 动态截取序列长度
        return x + Variable(self.pe[:, start: end], requires_grad=False)

**测试位置编码模块**

In [ ]:
# 初始化模块
dim_emb = 6
pos_encoder = PositionalEmbedding(dim_emb)

# 生成测试输入 (batch_size=2, seq_len=10)
x = torch.zeros(2, 10, dim_emb)

# 获取位置编码
x_with_pos = pos_encoder(x, 0, 10)

print(f'x_with_pos : \n{x_with_pos}')
print(f'x_with_pos size : \n{x_with_pos.size()}')

## Transformer 完整代码实现

---

有关 transformer 优化后的完整代码实现请参考同目录下的 transformer.py 文件。下面是相关测试：

In [ ]:
import torch
from torch import nn
from transformer import Transformer
import random

# 配置
# =====================================
encoder_num = 2  # 编码器数量
decoder_num = 5  # 解码器数量
vocab_size = 256 # 词表大小
dim_emb = 128    # 词向量维度
dim_head = 32    # 注意力头维度
head_num = 4     # 注意力头数

batch = 3             # 测试数据批数
input_seq_len = 12    # 测试输入数据长度
output_seq_len = 7    # 测试输出数据长度
output_masked_len = output_seq_len - 1 # 测试输出掩码长度
# =====================================

# 创建 transformer 网络
transformer = Transformer(encoder_num, decoder_num, vocab_size, dim_emb, dim_head, head_num)

def rand_seq_code(batch, seq_len):
    """
    随机序列编码

    :param batch: 批数
    :param seq_len: 序列长度
    :return: 随机序列编码
    """

    data = []
    for i in range(batch):
        bat_data = [random.randint(0, vocab_size) for _ in range(seq_len)]
        data.append(bat_data)
    return torch.tensor(data)

# 构建随机输入输出序列
input_seq = rand_seq_code(batch, input_seq_len)
input_mask = torch.stack([torch.arange(0,input_seq_len) for _ in range(batch)]) < 8

output_seq = rand_seq_code(batch, output_seq_len)
output_mask = torch.stack([torch.arange(0,output_seq_len) for _ in range(batch)]) < 3

# ===============
#  模拟第一次调用
# ===============

# transformer 输出
output, enc_output_cache ,kv_caches = transformer(
    input_seq,
    output_seq,
    input_mask = input_mask,
    output_mask = output_mask,
)

# 打印输出结果
print(f'output size  : {output.size()}')
print(f'kv cache len : {len(kv_caches)}')

# 预测下一个值
token = torch.argmax(output[:, -1, :], dim=-1)
print(f'prediction token : {token}')

# ===============
#  模拟第二次调用
# ===============

# transformer 输出
output_next, enc_out_next ,kv_caches_next = transformer(
    input_seq = None,
    output_seq = token.view(batch, 1),
    enc_output_cache = enc_output_cache,
    dec_kv_caches = kv_caches
)
token_next = torch.argmax(output_next[:, -1, :], dim=-1)
print(f'next prediction token : {token_next}')